# Accelerating End-to-End Data Science Workflows # 

## 03 - เส้นทางที่สั้นที่สุดจากแหล่งเดียวด้วย cuGraph ##

**cuGraph Single Source Shortest Path** 

**สารบัญ**
<br>
ในสมุดบันทึกนี้ เราจะใช้การวิเคราะห์กราฟที่เร่งด้วย GPU โดยใช้ cuGraph เพื่อระบุ **เส้นทางที่สั้นที่สุดจากจุดเชื่อมหนึ่งในเครือข่ายถนนไปยังทุกจุดเชื่อมอื่น** ทั้งตามระยะทาง ซึ่งเราจะสาธิตให้ดู และตามเวลา ซึ่งคุณจะต้องดำเนินการเอง คุณยังจะได้แสดงผลลัพธ์จากการค้นพบของคุณด้วย สมุดบันทึกนี้ครอบคลุมส่วนต่างๆ ดังต่อไปนี้:
1. [สภาพแวดล้อม (Environment)](#Environment)
2. [การโหลดข้อมูล (Loading Data)](#Loading-Data)
3. [การสร้างกราฟด้วย cuGraph (Construct Graph with cuGraph)](#Construct-Graph-with-cuGraph)
4. [การวิเคราะห์กราฟ (Analyzing the Graph)](#Analyzing-the-Graph)
5. [เส้นทางที่สั้นที่สุดจากแหล่งกำเนิดเดียว (Single Source Shortest Path)](#Single-Source-Shortest-Path)
6. [การวิเคราะห์กราฟด้วยน้ำหนักเวลา (Analyze a Graph with Time Weights)](#Analyze-a-Graph-with-Time-Weights)
    * [แบบฝึกหัดที่ 1 - ขั้นตอนที่ 1: การสร้างกราฟ (Exercise #1 - Step 1: Construct the Graph)](#Exercise-#1---Step-1:-Construct-the-Graph)
    * [แบบฝึกหัดที่ 2 - ขั้นตอนที่ 2: การหาระยะเวลาการเดินทางจากจุดเชื่อมหนึ่งไปยังจุดเชื่อมอื่นๆ ทั้งหมด (Exercise #2 - Step 2: Get Travel Times From a Node to All Others)](#Exercise-#2---Step-2:-Get-Travel-Times-From-a-Node-to-All-Others)
    * [การแสดงผลด้วยภาพของระยะเวลาการเดินทางของจุดเชื่อม (Visualize the Node Travel Times)](#Visualize-the-Node-Travel-Times)


## Environment ##

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import cudf
import cugraph as cg

import cuxfilter as cxf
from bokeh.palettes import Magma, Turbo256, Plasma256, Viridis256

## Loading Data ##

เราจะเริ่มต้นด้วยการโหลดข้อมูลกราฟถนนที่คุณเตรียมไว้สำหรับการสร้างกราฟด้วย cuGraph โดยที่ **`nodeid`** แบบยาวที่ไม่ซ้ำกันจะถูกแทนที่ด้วยจำนวนเต็มที่เรียบง่าย (และประหยัดหน่วยความจำ) ซึ่งเราเรียกว่า **`graph_id`**

In [ ]:
road_graph = cudf.read_csv('./data/road_graph.csv', dtype=['int32', 'int32', 'float32'])
road_graph.head()

ต่อไป เราจะโหลดข้อมูลที่พร้อมสำหรับสร้างกราฟ ซึ่งคุณได้เตรียมไว้แล้ว โดยข้อมูลนี้จะใช้ ระยะเวลาที่ใช้ในการเดินทาง เป็น ค่าน้ำหนัก (edge weight) ของเส้นเชื่อม (edge) ในกราฟ

In [ ]:
speed_graph = cudf.read_csv('./data/road_graph_speed.csv', dtype=['int32', 'int32', 'float32'])
speed_graph.head()

สุดท้ายนี้ เราจะนำเข้าชุดข้อมูล `road_nodes` ทั้งหมด ซึ่งจะนำไปใช้ในการแสดงผลด้านล่าง


In [ ]:
road_nodes = cudf.read_csv('./data/road_nodes.csv', dtype=['str', 'float32', 'float32', 'str'])
road_nodes = road_nodes.drop_duplicates() # again, some road nodes appeared on multiple map tiles in the original source
road_nodes.head()

In [ ]:
road_nodes.shape

In [ ]:
speed_graph.src.max()

## Construct Graph with cuGraph ##

เมื่อเราเตรียมข้อมูล `road_graph` เรียบร้อยแล้ว เราจะส่งข้อมูลนี้ไปยัง cuGraph เพื่อสร้าง **โครงสร้างข้อมูลกราฟ (graph data structure)** ซึ่งเราสามารถนำไปใช้ในการวิเคราะห์ที่รวดเร็วได้ ในการดำเนินการดังกล่าว ขั้นแรกเราจะใช้ cuGraph เพื่อสร้าง **อินสแตนซ์ (instance)** ของ `Graph` ขึ้นมา จากนั้นจึงส่งค่า **ต้นทางของเส้นเชื่อม (edge sources)**, **ปลายทางของเส้นเชื่อม (edge destinations)** และ **น้ำหนักของเส้นเชื่อม (edge weights)** ซึ่งปัจจุบันคือน้ำหนักของถนน ไปยังอินสแตนซ์นั้น


In [ ]:
G = cg.Graph()
%time G.from_cudf_edgelist(road_graph, source='src', destination='dst', edge_attr='length')

## Analyzing the Graph ##

ขั้นแรก เราจะตรวจสอบจำนวน **โหนด (nodes)** และ **เส้นเชื่อม (edges)** ในกราฟของเรา:

In [ ]:
G.number_of_nodes()

In [ ]:
G.number_of_edges()

เราสามารถวิเคราะห์ **ดีกรี (degrees)** ของโหนดในกราฟของเราได้ด้วย เช่นเคย เราคาดว่าทุกโหนดจะมีดีกรี 2 หรือสูงกว่า เนื่องจากเส้นเชื่อมแบบไม่มีทิศทาง (undirected edges) จะนับเป็นสองเส้นเชื่อม (หนึ่งเข้า, หนึ่งออก) สำหรับแต่ละโหนดที่เชื่อมต่อ

In [ ]:
deg_df = G.degree()
deg_df['degree'].describe()[1:]

เราคาดหวังว่าทุก ดีกรี (degree) จะต้องเป็น พหุคูณของ 2 ด้วยเหตุผลเดียวกัน เราจะตรวจสอบว่าไม่มีโหนดใดที่มี ดีกรีเป็นเลขคี่ (odd degrees) (นั่นคือ ดีกรีที่มีค่าเป็น 1 เมื่อหารด้วย 2)

In [ ]:
deg_df[deg_df['degree'].mod(2) == 1]

สังเกตเพื่อเป็นข้อมูลอ้างอิงว่าถนนบางสายวนจากจุดหนึ่งกลับมาที่จุดเดิม

In [ ]:
road_graph.loc[road_graph.src == road_graph.dst]

## Single Source Shortest Path ##

เพื่อสาธิตอัลกอริทึม **Single Source Shortest Path (SSSP)** เราจะเริ่มต้นที่โหนดที่มี **ระดับสูงสุด (highest degree)** ก่อนอื่น เราจะดึงค่า `graph_id` ของโหนดนั้น ซึ่งถูกรายงานโดยเมธอด `degree` ในชื่อ `vertex`

In [ ]:
demo_node = deg_df.nlargest(1, 'degree')
demo_node_graph_id = demo_node['vertex'].iloc[0]
demo_node_graph_id

ตอนนี้เราสามารถเรียกใช้ `cg.sssp` ได้ โดยส่งกราฟ `G` ทั้งหมด และ `graph_id` สำหรับจุดยอดที่เราเลือกเข้าไป การทำเช่นนี้จะคำนวณ **เส้นทางที่สั้นที่สุด** โดยใช้น้ำหนักความยาวถนนที่เราให้ไว้ ไปยัง **ทุกๆ โหนดอื่นในกราฟ** — เส้นทางนับล้านเส้นทาง จะคำนวณเสร็จสิ้นภายในไม่กี่วินาที

In [ ]:
%time shortest_distances_from_demo_node = cg.sssp(G, demo_node_graph_id)
shortest_distances_from_demo_node.head()

In [ ]:
# Limiting to those nodes that were connected (within ~4.3 billion meters because
# cg.sssp uses the max int value for unreachable nodes, such as those on different islands)
shortest_distances_from_demo_node['distance'].loc[shortest_distances_from_demo_node['distance'] < 2**32].describe()[1:]

## Analyze a Graph with Time Weights ##

สำหรับการฝึกฝนนี้ คุณจะวิเคราะห์กราฟถนนของสหราชอาณาจักร (GB) แต่คราวนี้ แทนที่จะใช้ระยะทางดิบเป็นน้ำหนักของถนน คุณจะใช้ **ระยะเวลาที่ใช้ในการเดินทาง** ไปตามถนนนั้นๆ

### Exercise #1 - Step 1: Construct the Graph ###

สร้างกราฟ cuGraph ชื่อ **G_ex** โดยใช้แหล่งที่มา (sources) และปลายทาง (destinations) ที่พบใน `speed_graph` พร้อมกับค่าความยาวเป็นวินาทีสำหรับน้ำหนัก (weights) ของเส้นเชื่อม (edges)

Click ... for solution. 

### Exercise #2 - Step 2: Get Travel Times From a Node to All Others ###

เลือก **node (จุดเชื่อม)** ใดจุดหนึ่ง แล้วคำนวณเวลาที่ใช้ในการเดินทางจากจุดนั้นไปยัง node อื่นๆ ทั้งหมดด้วยวิธี **SSSP (Shortest Single-Source Path)** โดยตั้งชื่อผลลัพธ์ว่า `ex_dist`

Click ... for solution. 

### Visualize the Node Travel Times ###

เพื่อสร้างกราฟแสดงเครือข่ายถนนโดยใช้เวลาเดินทางจากโหนดที่เลือก ขั้นแรกเราต้องจัดเรียงระยะทางที่คำนวณได้เมื่อสักครู่นี้ให้ตรงกับโหนดดั้งเดิมของมัน ในการทำเช่นนั้น เราจะใช้การแมปจากสตริง **`node_id`** ไปยังจำนวนเต็ม **`graph_id`** ของพวกมัน

In [ ]:
mapping = cudf.read_csv('./data/node_graph_map.csv')
mapping.head()

เราจะเห็นว่าอัลกอริทึม `sssp` ได้นำ **`graph_id`** ไปใส่ไว้ในคอลัมน์ **`vertex`** ดังนั้นเราจะทำการ **รวม (merge)** ข้อมูลโดยใช้คอลัมน์นั้น

In [ ]:
ex_dist.head()

In [ ]:
road_nodes = road_nodes.merge(mapping, on='node_id')
road_nodes = road_nodes.merge(ex_dist, left_on='graph_id', right_on='vertex')
road_nodes.head()

ต่อไป เราจะเลือกคอลัมน์ที่เราจะใช้สำหรับการแสดงผลข้อมูล (visualization)

สำหรับวัตถุประสงค์ในการปรับสเกลสี (color-scaling) เราจะกำจัด **โหนดที่ไม่สามารถเข้าถึงได้ (unreachable nodes)** ซึ่งมีค่าระยะทางที่สูงมาก และเราจะ **กลับค่าตัวเลขระยะทาง (invert the distance numbers)** เพื่อให้พิกเซลที่สว่างกว่าบ่งบอกถึงตำแหน่งที่ใกล้กว่า

In [ ]:
gdf = road_nodes[['east', 'north', 'distance']]
gdf = gdf[gdf['distance'] < 2**32]
gdf['distance'] = gdf['distance'].pow(1/2).mul(-1)

นอกเหนือจากนี้ การแสดงผลข้อมูลนี้จะคล้ายคลึงกับ **แผนภาพการกระจาย (scatter plots)** ที่เราสร้างขึ้นเพื่อแสดงภาพรวมของจำนวนประชากร แต่แทนที่จะระบายสีตามความหนาแน่นของจุด (point density) เหมือนในกรณีเหล่านั้น เราจะระบายสีตาม **เวลาเดินทางเฉลี่ย (mean travel time)** ไปยังโหนดต่างๆ ภายในพิกเซลนั้นๆ

In [ ]:
cxf_data = cxf.DataFrame.from_dataframe(gdf)

In [ ]:
heatmap_chart = cxf.charts.datashader.scatter(x='east', y='north', 
                                              # color_palette=Plasma256, # try also Turbo256, Viridis256, Magma, Plasma256
                                              # pixel_shade_type='linear', # can also be log, cbrt, linear
                                              aggregate_col='distance',
                                              aggregate_fn='mean',
                                              # point_shape='square',
                                              point_size=1)

In [ ]:
dash = cxf_data.dashboard([heatmap_chart], theme=cxf.themes.dark, data_size_widget=True)

dash.app()

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [สมุดบันทึกถัดไป](2-04_networkx_cugraph.ipynb) กันเลย

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">